In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
import itertools

%matplotlib inline
from finrl import config
from finrl import config_tickers
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.finrl_meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

/home/henry/miniconda3/envs/finrl/lib/python3.7/site-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [ ]:
import talib

In [1]:
from data_processor import ChinaStockDownloader


In [2]:
downloader = ChinaStockDownloader.ChinaStockDownloader(
        start_date="2019-01-01", end_date="2020-05-01", index="TEST"
    )

In [4]:
price_df = downloader.download_price()
price_df.head()

Load from downloaded data


,date,tic,open,high,low,close,amount
0,2019-01-02,sh.600000,9.74,9.79,9.58,9.70,2.296257e+08
1,2019-01-02,sh.600028,5.10,5.11,4.96,5.01,1.097685e+09
2,2019-01-02,sh.600030,16.01,16.01,16.01,16.01,0.000000e+00
3,2019-01-02,sh.600031,8.31,8.31,7.95,8.05,4.243270e+08
4,2019-01-03,sh.600000,9.70,9.82,9.66,9.81,1.819760e+08


In [5]:
price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1292 entries, 0 to 1291
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    1292 non-null   object 
 1   tic     1292 non-null   object 
 2   open    1292 non-null   float64
 3   high    1292 non-null   float64
 4   low     1292 non-null   float64
 5   close   1292 non-null   float64
 6   amount  1292 non-null   float64
dtypes: float64(5), object(2)
memory usage: 70.8+ KB


In [6]:
fe = FeatureEngineer(
    use_technical_indicator=True,
    tech_indicator_list=config.TECHNICAL_INDICATORS_LIST,
)

processed = fe.preprocess_data(price_df)
processed.head()

Successfully added technical indicators


,date,tic,open,high,low,close,amount,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2019-01-02,sh.600000,9.74,9.79,9.58,9.70,2.296257e+08,0.000000,9.910563,9.599437,100.0,66.666667,100.0,9.700,9.700
323,2019-01-02,sh.600028,5.10,5.11,4.96,5.01,1.097685e+09,0.000000,9.910563,9.599437,100.0,66.666667,100.0,5.010,5.010
646,2019-01-02,sh.600030,16.01,16.01,16.01,16.01,0.000000e+00,0.000000,9.910563,9.599437,100.0,66.666667,100.0,16.010,16.010
969,2019-01-02,sh.600031,8.31,8.31,7.95,8.05,4.243270e+08,0.000000,9.910563,9.599437,100.0,66.666667,100.0,8.050,8.050
1,2019-01-03,sh.600000,9.70,9.82,9.66,9.81,1.819760e+08,0.002468,9.910563,9.599437,100.0,66.666667,100.0,9.755,9.755


In [8]:
processed.sort_values(['tic', 'date'])

,date,tic,open,high,low,close,amount,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2019-01-02,sh.600000,9.74,9.79,9.58,9.70,2.296257e+08,0.000000,9.910563,9.599437,100.000000,66.666667,100.000000,9.700000,9.700000
1,2019-01-03,sh.600000,9.70,9.82,9.66,9.81,1.819760e+08,0.002468,9.910563,9.599437,100.000000,66.666667,100.000000,9.755000,9.755000
2,2019-01-04,sh.600000,9.73,10.00,9.70,9.96,2.689646e+08,0.007796,10.084357,9.562310,100.000000,100.000000,100.000000,9.823333,9.823333
3,2019-01-07,sh.600000,10.09,10.09,9.92,9.98,2.354402e+08,0.010751,10.127012,9.597988,100.000000,100.775194,100.000000,9.862500,9.862500
4,2019-01-08,sh.600000,10.03,10.03,9.91,9.96,1.505016e+08,0.011334,10.127112,9.636888,92.828009,65.920398,92.760890,9.882000,9.882000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1287,2020-04-24,sh.600031,19.27,19.60,19.11,19.20,1.021897e+09,0.583975,20.566859,16.980141,57.648231,69.917006,29.094536,17.883667,17.456167
1288,2020-04-27,sh.600031,19.50,19.55,18.92,19.37,1.174506e+09,0.559169,20.539347,17.219653,58.383887,64.187577,23.557929,17.970667,17.511833
1289,2020-04-28,sh.600031,19.48,19.81,19.20,19.78,1.317004e+09,0.566068,20.580781,17.395219,60.112504,75.018654,28.070522,18.111333,17.601000
1290,2020-04-29,sh.600031,18.90,19.58,18.75,19.47,1.392911e+09,0.540293,20.488058,17.704942,58.220955,54.871807,15.460129,18.229667,17.668833


In [44]:
# list_ticker = processed["tic"].unique().tolist()
# list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
# combination = list(itertools.product(list_date,list_ticker))

# processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
# processed_full = processed_full[processed_full['date'].isin(processed['date'])]
# processed_full = processed_full.sort_values(['date','tic'])

# processed_full = processed_full.fillna(0)

In [45]:
# processed_full.head()

,date,tic,open,high,low,close,amount,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2018-01-02,sh.600000,12.61,12.77,12.60,12.72,398614966.0,0.000000,12.774853,12.605147,0.000000,66.666667,100.000000,12.7200,12.7200
1,2018-01-03,sh.600000,12.73,12.80,12.66,12.66,480954809.0,-0.001346,12.774853,12.605147,0.000000,66.666667,100.000000,12.6900,12.6900
2,2018-01-04,sh.600000,12.70,12.73,12.62,12.66,353205838.0,-0.001715,12.749282,12.610718,0.000000,-100.000000,17.535545,12.6800,12.6800
3,2018-01-05,sh.600000,12.67,12.71,12.62,12.69,393058250.0,-0.000758,12.739946,12.625054,34.856700,-59.259259,17.535545,12.6825,12.6825
6,2018-01-08,sh.600000,12.69,12.71,12.63,12.68,397842209.0,-0.000620,12.731800,12.632200,31.116622,-50.314465,17.535545,12.6820,12.6820


In [9]:
fund_df = downloader.download_fundament()

login success!


logout success!


In [10]:
fund_df.head()

,tic,date,ROE,AssetStoEquity,Pnitoni,Nitogr,TaxBurden
0,sh.600000,2019-04-30,0.034284,13.371919,0.990015,0.331942,0.860641
1,sh.600000,2019-08-24,0.066587,13.428551,0.990590,0.332083,0.848056
2,sh.600000,2019-10-30,0.095587,12.929761,0.989481,0.333802,0.846423
3,sh.600000,2020-04-25,0.114901,12.965903,0.990001,0.312059,0.852314
4,sh.600000,2020-04-25,0.030746,12.668370,0.990359,0.316289,0.846615


In [11]:
fund_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tic             32 non-null     object
 1   date            32 non-null     object
 2   ROE             32 non-null     object
 3   AssetStoEquity  32 non-null     object
 4   Pnitoni         32 non-null     object
 5   Nitogr          32 non-null     object
 6   TaxBurden       32 non-null     object
dtypes: object(7)
memory usage: 1.9+ KB


In [21]:
processed = processed.merge(fund_df, how='left', on=['date', 'tic']).ffill(axis='rows')

In [23]:
processed.dropna()

,date,tic,open,high,low,close,amount,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,ROE,AssetStoEquity,Pnitoni,Nitogr,TaxBurden
311,2019-04-29,sh.600031,12.04,12.48,12.03,12.30,1.854152e+09,-0.010151,14.072677,11.705323,54.072306,-86.659664,13.231717,12.806667,11.603667,0.091727,2.241728,0.968844,0.156127,0.836110
312,2019-04-30,sh.600000,11.70,12.09,11.70,11.97,1.466715e+09,0.064680,12.062596,11.231404,59.600078,126.300366,24.865962,11.528000,11.397333,0.034284,13.371919,0.990015,0.331942,0.860641
313,2019-04-30,sh.600028,5.62,5.69,5.61,5.68,5.151295e+08,-0.044980,6.037154,5.643846,47.138425,-139.959603,18.546944,5.843667,5.873667,0.020251,2.335895,0.799253,0.025741,0.769304
314,2019-04-30,sh.600030,23.05,23.29,22.91,23.04,2.460179e+09,-0.182219,26.170809,22.652191,50.614181,-131.654425,24.423631,24.306333,23.205833,0.027375,4.264374,0.977183,0.414069,0.759405
315,2019-04-30,sh.600031,12.35,12.53,12.07,12.25,1.191862e+09,-0.028815,14.071009,11.636991,53.715481,-78.042417,14.529570,12.782000,11.656500,0.027375,4.264374,0.977183,0.414069,0.759405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1287,2020-04-29,sh.600031,18.90,19.58,18.75,19.47,1.392911e+09,0.540293,20.488058,17.704942,58.220955,54.871807,15.460129,18.229667,17.668833,0.048283,2.071733,0.990831,0.128224,0.838606
1288,2020-04-30,sh.600000,10.58,10.79,10.56,10.63,3.301575e+08,0.000749,10.544462,9.897538,50.440959,321.280603,32.344858,10.183000,10.552000,0.048283,2.071733,0.990831,0.128224,0.838606
1289,2020-04-30,sh.600028,4.41,4.46,4.38,4.46,5.295610e+08,-0.026558,4.590182,4.417818,44.085547,-129.473161,5.424623,4.491333,4.600500,-0.027257,2.454401,0.940567,-0.037861,0.736389
1290,2020-04-30,sh.600030,23.60,23.88,23.52,23.70,3.177410e+09,0.076595,23.885052,22.020948,52.255567,157.569020,19.078665,22.754000,23.376167,-0.027257,2.454401,0.940567,-0.037861,0.736389
